## Transferência de conhecimento
utilizando Resnet50 como base

In [1]:
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence

Para fins de demonstração vamos utilziar apenas 80 amostras, estamos utilizando uma arquitetura pesada e rodando pela CPU, o treinamento com o dataset completo tornaria inviável a demonstração.

In [2]:
with np.load("./data/mnist.npz", allow_pickle=True) as f:
    x_train, y_train = f['x_train'][:80], f['y_train'][:80]
    x_test, y_test = f['x_test'][:80], f['y_test'][:80]
print(f"{x_train.shape[0]} imagens de treino com resolução {x_train.shape[1]}x{x_train.shape[2]}")
print(f"shape: {x_train.shape}")


80 imagens de treino com resolução 28x28
shape: (80, 28, 28)


Apesar das nossas amostras estarem todas carregadas na memória, vamos criar um Batch Generator para fazer o pré-processamento das imagens.

In [3]:
class BatchGenerator(Sequence):
    def __init__(self, images, labels, batch_size=8):
        self._images = images
        self._labels = labels
        self._batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self._images) / self._batch_size))

    def __getitem__(self, idx):
        l_bound = idx * self._batch_size
        r_bound = (idx + 1) * self._batch_size

        if r_bound > len(self._images):
            r_bound = len(self._images)
            l_bound = r_bound - self._batch_size

        x_batch = np.zeros((r_bound - l_bound, 224, 224, 3))
        y_batch = np.zeros((r_bound - l_bound, 10))

        for instance_count, (image, label) in enumerate(
                zip(self._images[l_bound:r_bound], self._labels[l_bound:r_bound])):
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            image = cv2.resize(image, (224, 224))
            image = preprocess_input(image)
            x_batch[instance_count] = image
            y_batch[instance_count, label] = 1

        return x_batch, y_batch

In [4]:
train_generator = BatchGenerator(x_train, y_train)
val_generator = BatchGenerator(x_test, y_test)

## Modelo treinado sem o uso de pesos pré treinados

In [5]:
scratch_model = ResNet50()

x = scratch_model.layers[-2].output
x = Dense(10, activation="softmax")(x)
model_from_scratch = Model(scratch_model.input, x)
print(model_from_scratch.summary())

102967424/102967424 [==============================] - 36s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [6]:
model_from_scratch.compile(optimizer='sgd', loss='categorical_crossentropy')

model_from_scratch.fit(train_generator,
                       epochs=7,
                       validation_data=val_generator)


Epoch 1/7
10/10 [==============================] - 72s 6s/step - loss: 1.9092 - val_loss: 2.4762
Epoch 2/7
10/10 [==============================] - 81s 8s/step - loss: 0.5080 - val_loss: 2.1079
Epoch 3/7
10/10 [==============================] - 77s 8s/step - loss: 0.0665 - val_loss: 1.9543
Epoch 4/7
10/10 [==============================] - 61s 6s/step - loss: 0.0276 - val_loss: 1.7586
Epoch 5/7
10/10 [==============================] - 57s 6s/step - loss: 0.0189 - val_loss: 1.7010
Epoch 6/7
10/10 [==============================] - 49s 5s/step - loss: 0.0151 - val_loss: 1.6153
Epoch 7/7
10/10 [==============================] - 48s 5s/step - loss: 0.0127 - val_loss: 1.5406


## Modelo treinado utilizando pesos pré treinados

In [7]:
pretrained_model = ResNet50()
pretrained_model.load_weights("./data/resnet50.h5")

y = pretrained_model.layers[-2].output
y = Dense(10, activation="softmax")(y)
model = Model(pretrained_model.input, y)

for layer in model.layers[:-1]:
    layer.trainable = False
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [8]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

model.fit(train_generator,
          steps_per_epoch=5,
          epochs=7,
          validation_data=val_generator,
          validation_steps=10)

Epoch 1/7
5/5 [==============================] - 26s 3s/step - loss: 3.9739 - val_loss: 3.6279
Epoch 2/7
5/5 [==============================] - 12s 3s/step - loss: 3.2839 - val_loss: 2.8755
Epoch 3/7
5/5 [==============================] - 14s 3s/step - loss: 2.5513 - val_loss: 2.2111
Epoch 4/7
5/5 [==============================] - 13s 3s/step - loss: 1.5773 - val_loss: 2.2928
Epoch 5/7
5/5 [==============================] - 15s 4s/step - loss: 1.7356 - val_loss: 1.7256
Epoch 6/7
5/5 [==============================] - 12s 3s/step - loss: 1.4108 - val_loss: 1.5770
Epoch 7/7
5/5 [==============================] - 12s 3s/step - loss: 1.1829 - val_loss: 1.3458
